In [49]:
!pip install pandas numpy
!pip install matplotlib
import pandas as pd
import numpy as np
import os 
import re
import matplotlib.pyplot as plt

In [50]:
main_df = pd.read_csv('CJQ06.20260202004354.csv')
main_df = main_df.drop(["STATISTIC","C02480V03003","C02481V03160","TLIST(Q1)","UNIT"],axis=1)
pattern = r'\((\d{2})\)'
main_df["Year"] = main_df["Quarter"].str.slice(0,4)
main_df = main_df.drop("Quarter",axis=1)
main_df = main_df.groupby(
    ["Year", "Garda Division", "Type of Offence"],  
    as_index=False  
)["VALUE"].sum()
main_df = main_df.loc[main_df["Type of Offence"].str.contains(pattern, regex=True)]




C:\Users\downe\AppData\Local\Temp\ipykernel_26172\2037942219.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  main_df = main_df.loc[main_df["Type of Offence"].str.contains(pattern, regex=True)]


In [51]:
main_df

,Year,Garda Division,Type of Offence,VALUE
5,2003,Cavan/Monaghan Garda Division,"Attempts/threats to murder, assaults, harassme...",596.0
8,2003,Cavan/Monaghan Garda Division,Burglary and related offences (07),480.0
10,2003,Cavan/Monaghan Garda Division,Controlled drug offences (10),191.0
13,2003,Cavan/Monaghan Garda Division,Damage to property and to the environment off...,651.0
16,2003,Cavan/Monaghan Garda Division,Dangerous or negligent acts (04),866.0
...,...,...,...,...
54721,2025,Wicklow Garda Division,Road and traffic offences (14),688.0
54727,2025,Wicklow Garda Division,"Robbery, extortion and hijacking offences (06)",24.0
54728,2025,Wicklow Garda Division,Sexual offences (02),64.0
54731,2025,Wicklow Garda Division,Theft and related offences (08),1349.0


In [52]:
main_df["Type of Offence"] = main_df["Type of Offence"].str.replace(r"\s\(\d{2}\)$", "", regex=True)
crimes = main_df["Type of Offence"].unique()


In [ ]:
unique_division = main_df['Garda Division'].unique().tolist()
#replacing "/" with "-" to make writing filepaths easier
unique_division = [div.replace('/', '-') for div in unique_division]
main_df['Garda Division'] = main_df['Garda Division'].str.replace('/', '-')
#main_df.to_csv("cleaned_df.csv",index=False)
    


In [56]:
#seperate our big dataframe into a set of small dataframes based on each garda division
sub_dfs = {} # key = garda division, value = dataframe
for division in unique_division:
    sub_df = main_df.loc[main_df["Garda Division"]==division]
    sub_dfs[division] = sub_df


In [60]:
#writing each sub_df to its own individual csv file for future reference and ease
curr_dir = os.getcwd()
if not os.path.exists(os.path.join(curr_dir,"counties")):
    os.mkdir(os.path.join(curr_dir,"counties"),mode=0o755)

for division in sub_dfs.keys():
    division_path = os.path.join(curr_dir, "counties", division)
    os.makedirs(division_path, exist_ok=True)

    file_path = os.path.join(division_path, f"{division}_crime.csv")
    sub_dfs[division].to_csv(file_path, index=False)

In [61]:
dfs = []
for file in os.listdir("counties"):
    dfs.append(pd.read_csv(os.path.join(f"counties\\{file}",f"{file}_crime.csv")))
    

In [62]:
## Make a figure of 16 graphs, each containing information as to crime in each category over the years
county_crime_information = {}

for df in dfs:
    crimes_since_2003 = []
    for crime in crimes:
        y = df[df["Type of Offence"] == f"{crime}"]["VALUE"]
        x =  df[df["Type of Offence"] == f"{crime}"]["Year"]
        temp = [x,y]
        crimes_since_2003.append(temp)
    county_crime_information[df["Garda Division"][0]] = crimes_since_2003



In [64]:
for county in county_crime_information:

    fig, ax = plt.subplots(nrows = 4, ncols = 4,figsize=(16,16))
    fig.suptitle(f"{county}", fontsize=20, y=0.95) 


    for i,crime in enumerate(county_crime_information[county]):
        row = i//4
        col = i%4
        ax[row,col].plot(crime[0], crime[1], marker='o', linestyle='-', markerfacecolor='red')
        ax[row,col].set_title(f"{crimes[i]}",fontsize=10)
        ax[row,col].set_xlabel("Year")
        ax[row,col].set_ylabel("Number of Incidents")
        ax[row,col].grid(True)
        ax[row,col].tick_params(axis='x',rotation=45)


    plt.tight_layout(rect=[0, 0, 1, 0.95])  # leave space at the top for suptitle
    plt.savefig(f"counties/{county}/{county}_crimes.jpg")
    plt.close(fig)
